# Clean MNL script combining multiple PDAC samples
### Quantifies for the association between each LR and the four subtypes


In [77]:
import numpy as np
import csv
import pickle
import matplotlib
import math
import pandas as pd
import matplotlib
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

In [13]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [35]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
# coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
# with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
#     cell_barcode = pickle.load(file)


# Load subtype label
subtype_label_file='/Users/victoriagao/local_docs/schwartz_data/PDAC_64630_subtype.csv'
subtype_label=[]
with open(subtype_label_file) as file:
    csv_file = csv.reader(file, delimiter=",")
    for line in csv_file:
        subtype_label.append(line)

barcode_subtype=dict()
for i in range(1,len(subtype_label)):
    barcode_subtype[subtype_label[i][0]]= subtype_label[i][1]



# Load list of subtype label files
subtype_label_files = [
    '/Users/victoriagao/local_docs/schwartz_data/PDAC_64630_subtype.csv',
    '/Users/victoriagao/Documents/MSc/Schwartz_lab/experiment_data/Deisha/exp1_C1/cell_type_prediction.csv',
    '/Users/victoriagao/Documents/MSc/Schwartz_lab/experiment_data/Deisha/exp2_A1/cell_type_prediction.csv',
    '/Users/victoriagao/Documents/MSc/Schwartz_lab/experiment_data/Deisha/exp2_B1/cell_type_prediction.csv'
]

barcode_subtype_combined = {}

for file_index, subtype_label_file in enumerate(subtype_label_files, start=1):
    with open(subtype_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        next(csv_file)  # Skip the header row if there is one
        for line in csv_file:
            # Add a file-unique identifier to the barcode before storing it in the dictionary
            unique_barcode = f"{line[0]}-{file_index}"
            barcode_subtype_combined[unique_barcode] = line[1]


# # Load NEST output file into a 2D array
filenames = [
    "/Users/victoriagao/local_docs/NEST/output/From_Fatema/exp2_B1_top20percent.csv",
    "/Users/victoriagao/local_docs/NEST/output/From_Fatema/exp2_A1_top20percent.csv",
    "/Users/victoriagao/local_docs/NEST/output/From_Fatema/exp1_C1_top20percent.csv",
    "/Users/victoriagao/local_docs/NEST/output/From_Fatema/NEST_combined_output_PDAC_64630.csv"
]

# # Initialize an empty list to store DataFrames
dfs = []
for file_index, filename in enumerate(filenames, start=1):
    df = pd.read_csv(filename, sep=",")
    
    # Add file-unique identifier to the barcodes in "from_cell" and "to_cell" columns
    if "from_cell" in df.columns:
        df["from_cell"] = df["from_cell"].astype(str) + f"-{file_index}"
    if "to_cell" in df.columns:
        df["to_cell"] = df["to_cell"].astype(str) + f"-{file_index}"
    
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

# Preprocess NEST output df
df_processed = preprocessDf(combined_df)


In [36]:
combined_df

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,ATTATACTTTGCTCGT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,1.0,-1,327,1279,0.978623,TGFB1-ITGB5
1,ATTCAGGACCTATTTC-1-1,CTAGCCGATGTTATGA-1-1,TGFB1,ACVR1B,2.0,-1,330,608,0.978623,TGFB1-ACVR1B
2,ATTATGCCATAGGGAG-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,3.0,-1,328,673,0.978622,TGFB1-ITGB5
3,GCGGGAACCAGGCCCT-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,4.0,-1,811,673,0.978622,TGFB1-ITGB5
4,ACGCCAGATGATTTCT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,5.0,-1,123,1279,0.978622,TGFB1-ITGB5
...,...,...,...,...,...,...,...,...,...,...
1360309,AACGTCAGACTAGTGG-1-4,TTGGCTCGCATGAGAC-1-4,TGFB1,EGFR,NaN,17,31,1389,0.839991,TGFB1-EGFR
1360310,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,ITGB5,NaN,9,184,1394,0.829314,TGFB1-ITGB5
1360311,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,SDC2,NaN,9,184,1394,0.855152,TGFB1-SDC2
1360312,GAGAGGTGCATTCTGG-1-4,TTGTTTCCATACAACT-1-4,TGFB1,EGFR,NaN,2,715,1404,0.830097,TGFB1-EGFR


In [37]:
df_processed

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,ATTATACTTTGCTCGT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,1.0,-1,327,1279,0.978623,TGFB1-ITGB5
1,ATTCAGGACCTATTTC-1-1,CTAGCCGATGTTATGA-1-1,TGFB1,ACVR1B,2.0,-1,330,608,0.978623,TGFB1-ACVR1B
2,ATTATGCCATAGGGAG-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,3.0,-1,328,673,0.978622,TGFB1-ITGB5
3,GCGGGAACCAGGCCCT-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,4.0,-1,811,673,0.978622,TGFB1-ITGB5
4,ACGCCAGATGATTTCT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,5.0,-1,123,1279,0.978622,TGFB1-ITGB5
...,...,...,...,...,...,...,...,...,...,...
1360309,AACGTCAGACTAGTGG-1-4,TTGGCTCGCATGAGAC-1-4,TGFB1,EGFR,NaN,17,31,1389,0.839991,TGFB1-EGFR
1360310,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,ITGB5,NaN,9,184,1394,0.829314,TGFB1-ITGB5
1360311,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,SDC2,NaN,9,184,1394,0.855152,TGFB1-SDC2
1360312,GAGAGGTGCATTCTGG-1-4,TTGTTTCCATACAACT-1-4,TGFB1,EGFR,NaN,2,715,1404,0.830097,TGFB1-EGFR


In [38]:
# filter out the LR that only appeared once
df_processed_filtered = df_processed[df_processed['ligand-receptor'].duplicated(keep=False)] 
df_processed_filtered

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,ATTATACTTTGCTCGT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,1.0,-1,327,1279,0.978623,TGFB1-ITGB5
1,ATTCAGGACCTATTTC-1-1,CTAGCCGATGTTATGA-1-1,TGFB1,ACVR1B,2.0,-1,330,608,0.978623,TGFB1-ACVR1B
2,ATTATGCCATAGGGAG-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,3.0,-1,328,673,0.978622,TGFB1-ITGB5
3,GCGGGAACCAGGCCCT-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,4.0,-1,811,673,0.978622,TGFB1-ITGB5
4,ACGCCAGATGATTTCT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,5.0,-1,123,1279,0.978622,TGFB1-ITGB5
...,...,...,...,...,...,...,...,...,...,...
1360308,AACGTCAGACTAGTGG-1-4,TTGGCTCGCATGAGAC-1-4,TGFB1,ITGB5,NaN,17,31,1389,0.834446,TGFB1-ITGB5
1360309,AACGTCAGACTAGTGG-1-4,TTGGCTCGCATGAGAC-1-4,TGFB1,EGFR,NaN,17,31,1389,0.839991,TGFB1-EGFR
1360310,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,ITGB5,NaN,9,184,1394,0.829314,TGFB1-ITGB5
1360311,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,SDC2,NaN,9,184,1394,0.855152,TGFB1-SDC2


In [39]:
### Depends on the need for further eliminating LR pairs
lr_counts = df_processed_filtered['ligand-receptor'].value_counts()
threshold = lr_counts.quantile(0.90)  # gives the value at the 90th percentile
top_percent_lrs = lr_counts[lr_counts >= threshold].index
df_top_percent = df_processed_filtered[df_processed_filtered['ligand-receptor'].isin(top_percent_lrs)]
df_top_percent

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,ATTATACTTTGCTCGT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,1.0,-1,327,1279,0.978623,TGFB1-ITGB5
1,ATTCAGGACCTATTTC-1-1,CTAGCCGATGTTATGA-1-1,TGFB1,ACVR1B,2.0,-1,330,608,0.978623,TGFB1-ACVR1B
2,ATTATGCCATAGGGAG-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,3.0,-1,328,673,0.978622,TGFB1-ITGB5
3,GCGGGAACCAGGCCCT-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,4.0,-1,811,673,0.978622,TGFB1-ITGB5
4,ACGCCAGATGATTTCT-1-1,TTAATCAGTACGTCAG-1-1,TGFB1,ITGB5,5.0,-1,123,1279,0.978622,TGFB1-ITGB5
...,...,...,...,...,...,...,...,...,...,...
1360308,AACGTCAGACTAGTGG-1-4,TTGGCTCGCATGAGAC-1-4,TGFB1,ITGB5,NaN,17,31,1389,0.834446,TGFB1-ITGB5
1360309,AACGTCAGACTAGTGG-1-4,TTGGCTCGCATGAGAC-1-4,TGFB1,EGFR,NaN,17,31,1389,0.839991,TGFB1-EGFR
1360310,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,ITGB5,NaN,9,184,1394,0.829314,TGFB1-ITGB5
1360311,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,SDC2,NaN,9,184,1394,0.855152,TGFB1-SDC2


In [61]:
LR_df_filtered = df_top_percent.loc[df_top_percent["from_cell"].isin(barcode_subtype_combined.keys()) & df_top_percent["to_cell"].isin(barcode_subtype_combined.keys())]
LR_df_filtered

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
2,ATTATGCCATAGGGAG-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,3.0,-1,328,673,0.978622,TGFB1-ITGB5
3,GCGGGAACCAGGCCCT-1-1,CTTGTACTTGTTGACT-1-1,TGFB1,ITGB5,4.0,-1,811,673,0.978622,TGFB1-ITGB5
7,GCTCTCGGGTACCGAA-1-1,ATAGGTTGGGCAGATG-1-1,TGFB1,ITGB6,8.0,-1,835,262,0.978622,TGFB1-ITGB6
10,TCACCCTCTTAAGATT-1-1,AGTTCCTATTTATGTT-1-1,TGFB1,ENG,11.0,-1,1113,238,0.978620,TGFB1-ENG
12,GCTCTCGGGTACCGAA-1-1,TGATCGGTTTGACCCT-1-1,TGFB1,ENG,13.0,-1,835,1218,0.978622,TGFB1-ENG
...,...,...,...,...,...,...,...,...,...,...
1360304,CACAGCTAGGGAGTGA-1-4,TTGCTGAAGGAACCAC-1-4,PTPRF,MET,NaN,4,350,1384,0.724343,PTPRF-MET
1360305,TGTTTCGGTACTTCTC-1-4,TTGCTGAAGGAACCAC-1-4,PTPRF,MET,NaN,4,1305,1384,0.715855,PTPRF-MET
1360307,ATCCAGAGCAACAACC-1-4,TTGGATATCGTCTACG-1-4,TGFB3,SDC4,NaN,45,279,1388,0.787106,TGFB3-SDC4
1360310,AGATTATAGGACGTTT-1-4,TTGTAATCCGTACTCG-1-4,TGFB1,ITGB5,NaN,9,184,1394,0.829314,TGFB1-ITGB5


In [82]:
df_long

,from_cell,ligand-receptor
2,ATTATGCCATAGGGAG-1-1,TGFB1-ITGB5
3,GCGGGAACCAGGCCCT-1-1,TGFB1-ITGB5
7,GCTCTCGGGTACCGAA-1-1,TGFB1-ITGB6
10,TCACCCTCTTAAGATT-1-1,TGFB1-ENG
12,GCTCTCGGGTACCGAA-1-1,TGFB1-ENG
...,...,...
1360304,TTGCTGAAGGAACCAC-1-4,PTPRF-MET
1360305,TTGCTGAAGGAACCAC-1-4,PTPRF-MET
1360307,TTGGATATCGTCTACG-1-4,TGFB3-SDC4
1360310,TTGTAATCCGTACTCG-1-4,TGFB1-ITGB5


### Make feature matrix

In [88]:
# Processing the dataframe to get the counts
df_long = pd.concat([LR_df_filtered[['from_cell', 'ligand-receptor']], LR_df_filtered[['to_cell', 'ligand-receptor']].rename(columns={'to_cell': 'from_cell'})])
df_counts = df_long.groupby(['from_cell', 'ligand-receptor']).size().unstack(fill_value=0)
df_counts = df_counts.rename_axis('spot_barcode', axis='index')


# Creating X matrix
X = df_counts

In [90]:
type(df_counts)

pandas.core.frame.DataFrame

In [89]:
X

ligand-receptor,A2M-LRP1,A2M-TNFRSF14,ADAM9-ITGA6,AGT-ATP6AP2,AGT-CXCR4,AGT-F2R,AIMP1-RACK1,AIMP1-RRBP1,ANGPT2-ITGA5,ANGPTL2-ITGA5,...,VEGFB-FAT1,WNT5A-ANTXR1,WNT5A-CFTR,WNT5A-DAB2,WNT5A-FZD1,WNT5A-LDLR,WNT5A-M6PR,WNT5A-SCARB2,WNT5A-TFRC,YARS-RPSA
spot_barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
AAACACCAATAACTGC-1-3,0,0,0,0,0,0,0,0,0,1,...,0,5,0,0,0,5,0,0,0,0
AAACATTTCCCGGATT-1-2,3,4,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACATTTCCCGGATT-1-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCGGGTAGGTACC-1-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1-3,0,0,0,0,0,0,0,0,0,1,...,1,2,0,0,0,0,0,2,0,0
TTGTTTCATTAGTCTA-1-2,3,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTCATTAGTCTA-1-3,0,0,0,0,0,0,0,0,0,0,...,0,4,0,0,0,0,0,0,0,0


In [68]:
# Creating y vector
barcodes_in_X = df_counts.index.tolist()
y = pd.Series(barcodes_in_X).map(barcode_subtype_combined)

In [69]:
y

0         BasalB
1       ClassicB
2         BasalB
3       ClassicA
4         BasalB
          ...   
4145    ClassicB
4146      BasalA
4147    ClassicB
4148       Mixed
4149    ClassicA
Length: 4150, dtype: object

## sklearn.LogisticRegression

In [47]:
from numpy import mean
from numpy import std
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report
label_encoder = LabelEncoder()
from collections import Counter

In [66]:
# Y = label_encoder.fit_transform(y.index)
X = X.values

In [70]:
print(X.shape, y.shape)
print(Counter(y))

(4150, 468) (4150,)
Counter({'ClassicA': 1006, 'BasalB': 983, 'Mixed': 916, 'ClassicB': 864, 'BasalA': 381})


In [71]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X, y)
model

/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [72]:
# get the mapping of class labels to encoded numbers
class_labels = model.classes_

# display the mapping
for index, label in enumerate(class_labels):
    print(f"Class '{label}' is encoded as {index}.")

Class 'BasalA' is encoded as 0.
Class 'BasalB' is encoded as 1.
Class 'ClassicA' is encoded as 2.
Class 'ClassicB' is encoded as 3.
Class 'Mixed' is encoded as 4.


In [73]:
# model.decision_function(X).shape
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1) # evaluate the model and collect the scores
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

python(25708) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25709) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25710) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25711) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25712) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25713) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25714) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25715) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25716) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25717) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(25718) Malloc

Mean Accuracy: 0.543 (0.023)


/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

### Examining the model coefficients/weights

In [74]:
weights = model.coef_
# print("Feature Weights:\n", weights)
# weights.shape

In [75]:
#### Saving the coefficients to table outputs

# Feature names (LR pairs)
feature_names = df_counts.columns

# Class names (assuming model.classes_ gives you the class names in the order they're in the coefficients matrix)
class_names = model.classes_

# Dictionary to hold dataframes for each class
coefficients_dfs = {}

for index, class_name in enumerate(class_names):
    # Create a DataFrame for each class
    class_df = pd.DataFrame(weights[index], index=feature_names, columns=["Coefficient"])
    
    # Sort the DataFrame by the absolute values of the coefficients for better interpretability
    class_df = class_df.reindex(class_df.Coefficient.abs().sort_values(ascending=False).index)
    
    # Save the DataFrame in the dictionary
    coefficients_dfs[class_name] = class_df

    # save to CSV
    class_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/MNL_subtype_coefficients/aggregated_coefficients_for_{class_name}.csv")


### Computing p-values for coefficients calculated

In [78]:
## get stand errors for each coefficient


n_iterations = 1000
n_classes = model.coef_.shape[0] 
n_features = model.coef_.shape[1]

# empty array to hold the bootstrap coefficients
bootstrap_coefs = np.zeros((n_iterations, n_classes, n_features))

model_bootstrap = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# bootstrap
# for i in range(n_iterations):
#     X_sample, y_sample = resample(X, y)
#     model_bootstrap.fit(X_sample, y_sample)
#     bootstrap_coefs[i] = model_bootstrap.coef_

for i in range(n_iterations):
    try:
        X_sample, y_sample = resample(X, y)
        model_bootstrap.fit(X_sample, y_sample)
        if model_bootstrap.coef_.shape == (n_classes, n_features):
            bootstrap_coefs[i] = model_bootstrap.coef_
        else:
            # Skip this iteration if the shape does not match
            continue
    except ValueError as e:
        print(f"Skipping iteration {i} due to error: {e}")
        continue


# standard errors for each coefficient
bootstrap_standard_errors = np.std(bootstrap_coefs, axis=0)

# standard errors for each coefficient across classes
bootstrap_standard_errors_per_feature = np.std(bootstrap_coefs, axis=(0, 1))

print("Bootstrap Standard Errors (per class and feature):", bootstrap_standard_errors)
print("Bootstrap Standard Errors (per feature across all classes):", bootstrap_standard_errors_per_feature)


/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

Bootstrap Standard Errors (per class and feature): [[0.04588466 0.05532772 0.04293384 ... 0.04746914 0.01469746 0.07516062]
 [0.03640427 0.04665289 0.05248801 ... 0.06026739 0.03277246 0.07401176]
 [0.06722597 0.07170586 0.07097184 ... 0.0729669  0.05652544 0.08330154]
 [0.05701179 0.07895825 0.07266425 ... 0.07911273 0.05617358 0.04318186]
 [0.03044429 0.04459206 0.06686262 ... 0.08450528 0.04151183 0.0817243 ]]
Bootstrap Standard Errors (per feature across all classes): [0.07551032 0.07758076 0.08069444 0.0794217  0.07470208 0.10926268
 0.1157903  0.05438199 0.07244396 0.10628223 0.0781777  0.08765081
 0.10094613 0.07960012 0.06595402 0.07293894 0.08244225 0.01594489
 0.08603878 0.06364604 0.05401528 0.13039295 0.04779552 0.11230597
 0.09879941 0.1491008  0.07408754 0.1004759  0.1628288  0.14817108
 0.09922831 0.17824478 0.08283957 0.07298371 0.04695497 0.0232015
 0.02195544 0.04163208 0.02356848 0.04335743 0.10223805 0.02408012
 0.05649269 0.04168222 0.03676225 0.02646527 0.03742035

/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [79]:
# weights.shape
bootstrap_standard_errors.shape
# bootstrap_standard_errors_per_feature.shape

(5, 468)

In [80]:
# wald statistics for p-values
from scipy.stats import chi2

wald_stats = (weights / bootstrap_standard_errors) ** 2
p_values = [1 - chi2.cdf(w, 1) for w in wald_stats]

for i, p in enumerate(p_values):
    print(f'Coefficient {i}: p-value = {p}')

Coefficient 0: p-value = [8.70637677e-01 5.96462481e-01 7.35449119e-01 5.36363897e-01
 1.54203674e-01 2.55723562e-01 4.03547808e-01 6.73446772e-01
 6.47311858e-01 6.01860000e-01 2.89140464e-01 1.54828645e-01
 5.01576375e-01 2.62918890e-01 7.36387652e-02 7.74616772e-01
 4.05303707e-01 8.38474138e-01 9.83076699e-01 6.01807923e-01
 5.86922571e-01 3.78470740e-01 4.55181009e-01 4.14403231e-01
 2.84355893e-01 3.89242880e-01 5.20015635e-01 2.17627834e-01
 1.58680836e-01 1.31384754e-03 5.39241321e-01 9.05763072e-01
 2.65548579e-01 7.12494093e-01 3.54616309e-01 9.76745663e-01
 8.30387032e-01 4.40316931e-01 9.42422627e-01 5.05386306e-01
 7.95395664e-01 6.64878983e-01 9.77238516e-01 5.20220159e-01
 2.62886964e-01 8.90086312e-01 9.32057296e-01 7.28217717e-01
 1.58446256e-01 9.83294184e-02 7.18899783e-01 1.32478027e-01
 5.20939488e-01 1.04602627e-01 4.69636299e-01 1.10944215e-01
 5.26065315e-01 1.68315006e-01 9.84929167e-01 5.28162503e-01
 3.36135709e-01 7.63656669e-01 3.31000183e-01 2.10264462e-01

In [81]:
for i, p in enumerate(p_values):
    # Create a DataFrame for each class
    p_value_df = pd.DataFrame(p)

    # save to CSV
    p_value_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/MNL_subtype_coefficients/agg_pvalues_for_subtype{i}_coefficient.csv")



### Trying to put everthing into functions

In [91]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from scipy.stats import chi2

def fit_model(X, y):
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    model.fit(X, y)
    return model

def bootstrap_coefficients(X, y, n_iterations, model):
    n_classes, n_features = model.coef_.shape
    bootstrap_coefs = np.zeros((n_iterations, n_classes, n_features))
    for i in range(n_iterations):
        try:
            X_sample, y_sample = resample(X, y)
            model.fit(X_sample, y_sample)
            if model.coef_.shape == (n_classes, n_features):
                bootstrap_coefs[i] = model.coef_
        except ValueError as e:
            continue
    return np.std(bootstrap_coefs, axis=0)

def calculate_p_values(weights, bootstrap_standard_errors):
    wald_stats = (weights / bootstrap_standard_errors) ** 2
    p_values = 1 - chi2.cdf(wald_stats, 1)
    return p_values



In [92]:
# Assuming X and y are defined
model = fit_model(X, y)
bootstrap_standard_errors = bootstrap_coefficients(X, y, 1000, model)

feature_names = df_counts.columns
class_names = model.classes_
coefficients_dfs = {}

for index, class_name in enumerate(class_names):
    coefs = model.coef_[index]
    p_values = calculate_p_values(coefs, bootstrap_standard_errors[index])
    
    # Create a DataFrame for coefficients and p-values
    class_df = pd.DataFrame({
        "Coefficient": coefs,
        "P_Value": p_values
    }, index=feature_names)
    
    # Sort by the absolute values of the coefficients
    class_df = class_df.reindex(class_df.Coefficient.abs().sort_values(ascending=False).index)
    
    # Save the DataFrame
    coefficients_dfs[class_name] = class_df
    
    # Save to CSV
    class_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/MNL_subtype_coefficients/new_aggregated_coefficients_pvalues_for_{class_name}.csv")



/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html